# *An Introduction to Stochastic Processes in Physics* by Don S. Lemons

This text provides an introduction to stochastic processes and the necessary mathematical tools of probability, random walks, Brownian motion, and Weiner and Ornstein-Uhlenbeck processes.

## Dependencies

In [13]:
// NuGet packages
#r "nuget: plotly.net"
#r "nuget: Plotly.NET.Interactive"

Installed Packages plotly.net, 4.1.0 Plotly.NET.Interactive, 4.2.1

In [14]:
// Import declarations
open Plotly.NET

## Chapter 1: Random Variables

In [153]:
open System.Collections.Generic
open System.Runtime.CompilerServices

// Declaring "interfaces with static abstract methods" is an advanced feature.
#nowarn "3535"
//#nowarn "3536"

[<Interface>]
type IRandomVariable<'T when IRandomVariable<'T>> =
    abstract member Probability: float
    abstract member ConvertSampleToMeasurable: float
    static abstract member Possibilities: 'T list

let X (randomVariable: #IRandomVariable<'T>) =
    randomVariable.ConvertSampleToMeasurable
    
let P (randomVariable: 'T when IRandomVariable<'T>) =
    randomVariable.Probability

[<AutoOpen>]
module Extension =
    type IRandomVariable<'T when IRandomVariable<'T>> with
        static member ExpectedValue =
            IRandomVariable<'T>.Possibilities
            |> List.map (fun x_i -> X(x_i) * P(x_i))
            |> List.sum

// [<Extension>]
// type IRandomVariableExtension<'T> =

//     [<Extension>]
//     static member ExpectedValue =
//         ('T :> IRandomVariable<'T>).Possibilities
//         |> List.map (fun x_i -> X(x_i) * P(x_i))
//         |> List.sum



let expectedValue (randomVariable: 'T when IRandomVariable<'T>) =
    'T.Possibilities
    |> List.map (fun x_i -> X(x_i) * P(x_i))
    |> List.sum

In [161]:
/// Represents random variable for the result of a coint toss
type CoinToss =
    | Heads
    | Tails

    interface IRandomVariable<CoinToss> with
        member x.Probability = 0.50
        
        member x.ConvertSampleToMeasurable =
            match x with
            | Heads -> 1.0
            | Tails -> 0.0
        
        static member Possibilities =
            [Heads; Tails]

In [163]:
IRandomVariable<CoinToss>.ExpectedValue

Error: input.fsx (1,1)-(1,40) typecheck error Type constraint mismatch when applying the default type 'IRandomVariable<'a>' for a type inference variable. The types ''a' and 'IRandomVariable<'a>' cannot be unified. Consider adding further type constraints

In [17]:
/// Toss a coin given a random number generator that is assumed to be uniformly distributed
let coinToss (random: System.Random) =
    match random.NextInt64(2) with
    | 0L     -> Tails
    | 1L | _ -> Heads
    // The wildcard case won't happen since NextInt64(2) gives either 0L or 1L

/// Generate a list of number of coin tosses and the frequency of heads at
/// each given coin toss
let frequencyOfHeads n : (int * float) list =
    let random = System.Random()

    let frequency (frequencies, numberOfHeads) coinTossNumber =
        let newNumberOfHeads =
            match coinToss random with
            | Heads -> numberOfHeads + 1
            | Tails -> numberOfHeads
        ((coinTossNumber, float newNumberOfHeads / float coinTossNumber) :: frequencies, newNumberOfHeads)

    [1..n]
    |> List.fold frequency ([], 0)
    |> fst
    |> List.rev

In [18]:
// An example coin toss
coinToss (System.Random())

Heads

In [19]:
frequencyOfHeads 10

index value 0 (1, 1) Item1 1 Item2 1 1 (2, 1) Item1 2 Item2 1 2 (3, 1) Item1 3 Item2 1 3 (4, 1) Item1 4 Item2 1 4 (5, 0.8) Item1 5 Item2 0.8 5 (6, 0.8333333333333334) Item1 6 Item2 0.8333333333333334 6 (7, 0.8571428571428571) Item1 7 Item2 0.8571428571428571 7 (8, 0.75) Item1 8 Item2 0.75 8 (9, 0.6666666666666666) Item1 9 Item2 0.6666666666666666 9 (10, 0.7) Item1 10 Item2 0.7

In [34]:
let n = 10_000
let (x, y) = frequencyOfHeads n |> List.unzip
[ Chart.Line([(1, 0.5); (n, 0.5)], Name = "P(1)", MarkerColor  = Color.fromKeyword DimGray);
  Chart.Spline(x, y, Name = "Frequency of heads", MarkerColor  = Color.fromKeyword DodgerBlue);
]
|> Chart.combine
|> Chart.withTitle("Frequency of heads in coin tosses")
|> Chart.withSize(700, 500)
|> Chart.withXAxisStyle("n", AxisType = StyleParam.AxisType.Log)
|> Chart.withYAxisStyle("f(1)", MinMax = (0, 1))

<!-- Plotly chart will be drawn inside this DIV -->

In [22]:
type UnbiasedDie =
    | One
    | Two
    | Three
    | Four
    | Five
    | Six

    interface IRandomVariable<UnbiasedDie> with
        member x.ConvertSampleToMeasurable =
            match x with
            | One   -> 1.0
            | Two   -> 2.0
            | Three -> 3.0
            | Four  -> 4.0
            | Five  -> 5.0
            | Six   -> 6.0
        member x.Probability = 1.0 / 6.0
        static member Possibilities = [One; Two; Three; Four; Five; Six]

Error: input.fsx (9,15)-(9,30) typecheck error The type 'IRandomVariable' is not defined.
input.fsx (9,15)-(9,43) typecheck error The type 'obj' is not an interface type
input.fsx (9,15)-(9,30) typecheck error The type 'IRandomVariable' is not defined.
input.fsx (9,15)-(9,30) typecheck error The type 'IRandomVariable' is not defined.
input.fsx (10,18)-(10,43) typecheck error No abstract property was found that corresponds to this override
input.fsx (18,18)-(18,29) typecheck error No abstract property was found that corresponds to this override
input.fsx (19,23)-(19,36) typecheck error No abstract property was found that corresponds to this override

In [23]:
expectedValue One

Error: input.fsx (1,1)-(1,14) typecheck error The value or constructor 'expectedValue' is not defined.